In [1]:
import pandas as pd

import tweepy
import praw
import config

from bots.twitter import TwitterBot
from bots.reddit import RedditBot

2023-03-17 16:55:27,872 - INFO     | config     | Loading environment variables
2023-03-17 16:55:27,874 - WARNING  | config     | Directory data already exists


## Twitter

In [2]:
twitter_bot = TwitterBot(bearer=config.TWTR_BEARER_TOKEN,
                         api=config.TWTR_API,
                         api_secret=config.TWTR_API_SECRET,
                         access=config.TWTR_ACCESS_TOKEN,
                         access_secret=config.TWTR_ACCESS_TOKEN_SECRET)

In [3]:
query = 'entity:"Nike" -is:retweet lang:en'

In [4]:
tweets = twitter_bot.get_recent_tweets(query=query, limit=10)

In [33]:
tweets_df = pd.DataFrame.from_dict(tweets)
tweets_df.head()

,id,created_at,user,text,entities
0,1636757558605017091,2023-03-17 15:53:06+00:00,112752849,"In case you've been living under a rock, @Nike...","{Technology, Nike, Fashion, Blockchain, Retail..."
1,1636744910627016704,2023-03-17 15:02:50+00:00,574186881,AD: Jordan 1 High Golf ‘White/Midnight Navy’ \...,"{Sports & Fitness Business, Sports, Nike, Jord..."
2,1636744772848349184,2023-03-17 15:02:17+00:00,1162137647269105667,hmmm. Great question. I'd go Clone-X because t...,"{Fashion & beauty, Digital asset industry, Tec..."
3,1636744592283320320,2023-03-17 15:01:34+00:00,3009914161,Air Jordan 1 High Golf “Midnight Navy” restock...,"{Sports & Fitness Business, Sports, Nike, Jord..."
4,1636744521844416512,2023-03-17 15:01:17+00:00,2698270332,Ad: FULL RESTOCK\nAir Jordan 1 High Golf ‘Whit...,"{Fashion & beauty, Sports & Fitness Business, ..."


In [34]:
tweets_df['created_at'] = tweets_df['created_at'].dt.strftime('%Y-%m-%d')

In [35]:
tweets_df

,id,created_at,user,text,entities
0,1636757558605017091,2023-03-17,112752849,"In case you've been living under a rock, @Nike...","{Technology, Nike, Fashion, Blockchain, Retail..."
1,1636744910627016704,2023-03-17,574186881,AD: Jordan 1 High Golf ‘White/Midnight Navy’ \...,"{Sports & Fitness Business, Sports, Nike, Jord..."
2,1636744772848349184,2023-03-17,1162137647269105667,hmmm. Great question. I'd go Clone-X because t...,"{Fashion & beauty, Digital asset industry, Tec..."
3,1636744592283320320,2023-03-17,3009914161,Air Jordan 1 High Golf “Midnight Navy” restock...,"{Sports & Fitness Business, Sports, Nike, Jord..."
4,1636744521844416512,2023-03-17,2698270332,Ad: FULL RESTOCK\nAir Jordan 1 High Golf ‘Whit...,"{Fashion & beauty, Sports & Fitness Business, ..."
5,1636743542931615745,2023-03-17,574186881,AD: All sizes showing on SNKRS Desktop\nNike A...,"{SNKRS, Fashion & beauty, Sneakers, Nike, Fash..."
6,1636739872000966656,2023-03-17,373165951,Ad: Air Max 1 Premium Corduroy 'Baltic Blue' s...,"{SNKRS, Sneakers, Textile, Apparel & Luxury Go..."
7,1636726831633031171,2023-03-17,574186881,AD: ALMOST LIVE\nNike Air Max 1 PRM Corduroy ‘...,"{SNKRS, Fashion & beauty, Fashion Brand, Sneak..."
8,1636719879611985920,2023-03-17,373165951,Ad: Nike Air Max 1 PRM Corduroy 'Baltic Blue' ...,"{Fashion & beauty, Sneakers, Nike, Fashion, Te..."
9,1636718202938961921,2023-03-17,14462333,@Pan_the_Man Hey Bob. Sorry to hear you're hav...,"{Nike, Fashion, Textile, Apparel & Luxury Good..."


## Reddit

In [8]:
reddit_bot = RedditBot(client_id=config.REDDIT_APP_ID,
                       client_secret=config.REDDIT_SECRET,
                       redirect_url=config.REDDIT_REDIRECT_URL,
                       user_agent=config.REDDIT_USER_AGENT)

In [11]:
d = {}
for sub in reddit_bot.client.subreddit('MachineLearning').top(time_filter='all', limit=50):
    d['id'] = sub.id
    d['']

2023-03-17 16:58:22,547 - DEBUG    | prawcore   | Fetching: GET https://oauth.reddit.com/r/MachineLearning/top
2023-03-17 16:58:22,549 - DEBUG    | prawcore   | Data: None
2023-03-17 16:58:22,550 - DEBUG    | prawcore   | Params: {'t': 'all', 'limit': 50, 'raw_json': 1}
2023-03-17 16:58:24,262 - DEBUG    | prawcore   | Response: 200 (84759 bytes)


gh1dj9
kuc6tz
g7nfvb
lui92h
ohxnts
n2f0ld
vkxsf2
hiv3vf
m554cq
j0oyk6
leq2kf
klbvaw
ybnnra
gc2wo9
zhrgln
pmqtj9
juv419
hciw10
10ys3md
s4tu5x
xtxe6f
dh2xfs
jm86z9
ma8xbq
748cco
o843t5
g7wvpb
11sboh1
i1aafb
kythnj
kp5pxi
sfbtds
mouyp0
wmypmh
jcuch4
5z8110
xyxe8w
rdsepx
t7qe6b
o3804y
qo4kp8
jybogw
ia2aob
m47an8
10tovhn
10ch0kw
thsx8t
j0btow
yaqlvi
uks8zr


In [41]:
t = []
for post in reddit_bot.client.subreddit('TalesFromTheCustomer').top(time_filter='all', limit=50):
    d = {}
    d['id'] = post.id
    d['text'] = post.title + '\n' + post.selftext
    t.append(d)

2023-03-17 17:44:56,654 - DEBUG    | prawcore   | Fetching: GET https://oauth.reddit.com/r/TalesFromTheCustomer/top
2023-03-17 17:44:56,658 - DEBUG    | prawcore   | Data: None
2023-03-17 17:44:56,658 - DEBUG    | prawcore   | Params: {'t': 'all', 'limit': 50, 'raw_json': 1}
2023-03-17 17:44:57,893 - DEBUG    | prawcore   | Response: 200 (93244 bytes)


In [48]:
comments = []
for post in t:
    post_id = post['id']
    
    submission = reddit_bot.client.submission(post_id)
    
    submission.comments.replace_more(limit=None)
    for comment in submission.comments.list():
        comments.append(comment)
    break

2023-03-17 17:48:25,526 - DEBUG    | prawcore   | Fetching: GET https://oauth.reddit.com/comments/b547zx/
2023-03-17 17:48:25,529 - DEBUG    | prawcore   | Data: None
2023-03-17 17:48:25,530 - DEBUG    | prawcore   | Params: {'limit': 2048, 'sort': 'confidence', 'raw_json': 1}
2023-03-17 17:48:29,538 - DEBUG    | prawcore   | Response: 200 (113855 bytes)
2023-03-17 17:48:29,600 - DEBUG    | prawcore   | Fetching: POST https://oauth.reddit.com/api/morechildren/
2023-03-17 17:48:29,600 - DEBUG    | prawcore   | Data: [('api_type', 'json'), ('children', 'ejbcrcz,ejcw2h1,ejbrkso,ejcw77v,ejbd5ar,ejc69zr,ejb95zu,ejcsfq5,ejb7b0l,ejblysc,ejcxo9v,ejcbxgn,ejb7pux,ejbbezy,ejcrdwv,ejcwcds,ejcft0x,ejbuto2,ejclvhw,ejbgurc,ejb6x4x,ejcc30j,ejcxbnt,ejcdxot,ejco9er,ejcsrf8,ejbjcd1,ejc5nop,ejb70pl,ejbtl24,ejbxxwg,ejbyeb5,ejblnqc,ejcss86,ejbu60c,ejcd6ul,ejctkhc,ejbv3zl,ejbdqsz,ejcwt8e,ejbus2x,ejbj2g9,ejcmgvw,ejbtqm5,ejcso72'), ('link_id', 't3_b547zx'), ('sort', 'confidence')]
2023-03-17 17:48:29,601 - DEB

2023-03-17 17:48:36,519 - DEBUG    | prawcore   | Fetching: POST https://oauth.reddit.com/api/morechildren/
2023-03-17 17:48:36,521 - DEBUG    | prawcore   | Data: [('api_type', 'json'), ('children', 'ejblnux'), ('link_id', 't3_b547zx'), ('sort', 'confidence')]
2023-03-17 17:48:36,522 - DEBUG    | prawcore   | Params: {'raw_json': 1}
2023-03-17 17:48:36,726 - DEBUG    | prawcore   | Response: 200 (816 bytes)
2023-03-17 17:48:36,729 - DEBUG    | prawcore   | Fetching: POST https://oauth.reddit.com/api/morechildren/
2023-03-17 17:48:36,732 - DEBUG    | prawcore   | Data: [('api_type', 'json'), ('children', 'ejc6ul9'), ('link_id', 't3_b547zx'), ('sort', 'confidence')]
2023-03-17 17:48:36,733 - DEBUG    | prawcore   | Params: {'raw_json': 1}
2023-03-17 17:48:36,925 - DEBUG    | prawcore   | Response: 200 (1081 bytes)
2023-03-17 17:48:36,927 - DEBUG    | prawcore   | Fetching: POST https://oauth.reddit.com/api/morechildren/
2023-03-17 17:48:36,928 - DEBUG    | prawcore   | Data: [('api_type

I’ve had a bad experience with them also. Nothing like what happened to you but an item was missing from my order and boobs blamed restaurant and restaurant blamed boobs. The in app chat with boobs got me nowhere and they kept marking the issue as “solved” and despite my reopening and replying they just stopped responding. I was pissed. If they had offered to refund even just $5 off my order I would a have been happy but they just shrugged and ignored me. 
Almost the same thing happened with me. I got a notification that my driver couldn’t find my address. I looked outside and she was sitting in her car right outside my apartment. Clearly insight of my door with the number printed on it. She was too lazy to get out of her actual car so I texted her and she said she was lost, she watched me go outside and up to her car and wouldn’t even talk to me. Lol I have uber 
Believe it or not, this has happened to me too. Woman ran off with my pizza after making me wait an extra half hour. Contac